In [ ]:
import zipfile
import os

zip_file_path = '/content/text_sorted_confidence.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/personal_project')  # Unzip to the specified directory


In [ ]:
import os
from collections import defaultdict

# Define the range of years
years = range(1915, 1932)

# Initialize data structures
yearly_counts = defaultdict(int)
yearly_counts_09 = defaultdict(int)
word_counts = defaultdict(int)
word_counts_09 = defaultdict(int)
min_word_counts = defaultdict(lambda: float('inf'))
max_word_counts = defaultdict(lambda: float('-inf'))
min_word_counts_09 = defaultdict(lambda: float('inf'))
max_word_counts_09 = defaultdict(lambda: float('-inf'))

# Root directory path
root_path = '/content/personal_project/text_sorted_confidence'

# Iterate through all subfolders
for folder in os.listdir(root_path):
    folder_path = os.path.join(root_path, folder)

    if os.path.isdir(folder_path):
        # Iterate through all txt files in the folder
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.txt'):
                # Extract the year
                year_prefix = file_name[:2]
                year = 1900 + int(year_prefix)

                if year in years:
                    file_path = os.path.join(folder_path, file_name)
                    with open(file_path, 'r', encoding='utf-8') as file:
                        content = file.read()
                        word_count = len(content)  # Assume the text content is already tokenized

                    # Increase the number of files and word count for the corresponding year
                    yearly_counts[year] += 1
                    word_counts[year] += word_count

                    # Update the minimum and maximum word count
                    if word_count < min_word_counts[year]:
                        min_word_counts[year] = word_count
                    if word_count > max_word_counts[year]:
                        max_word_counts[year] = word_count

                    # If inside the 0.9 folder, increase the 0.9 file count and word count for the corresponding year
                    if folder == '0.9':
                        yearly_counts_09[year] += 1
                        word_counts_09[year] += word_count

                        # Update the minimum and maximum word count for the 0.9 folder
                        if word_count < min_word_counts_09[year]:
                            min_word_counts_09[year] = word_count
                        if word_count > max_word_counts_09[year]:
                            max_word_counts_09[year] = word_count

# Output the results
for year in years:
    total_count = yearly_counts[year]
    count_09 = yearly_counts_09[year]
    total_word_count = word_counts[year]
    word_count_09 = word_counts_09[year]
    min_word_count = min_word_counts[year] if min_word_counts[year] != float('inf') else 0
    max_word_count = max_word_counts[year] if max_word_counts[year] != float('-inf') else 0
    min_word_count_09 = min_word_counts_09[year] if min_word_counts_09[year] != float('inf') else 0
    max_word_count_09 = max_word_counts_09[year] if max_word_counts_09[year] != float('-inf') else 0
    proportion_09 = count_09 / total_count if total_count > 0 else 0
    word_proportion_09 = word_count_09 / total_word_count if total_word_count > 0 else 0

    print(f"Year: {year}, Total Count: {total_count}, 0.9 Count: {count_09}, 0.9 Proportion: {proportion_09:.2%}")
    print(f"Total Word Count: {total_word_count}, 0.9 Word Count: {word_count_09}, 0.9 Word Proportion: {word_proportion_09:.2%}")
    print(f"Min Word Count: {min_word_count}, Max Word Count: {max_word_count}")
    print(f"Min Word Count 0.9: {min_word_count_09}, Max Word Count 0.9: {max_word_count_09}")
    print("-" * 50)


In [ ]:
import os
from collections import defaultdict

# Define the directory path
main_directory_09 = '/content/personal_project/data'
sub_dirs = [main_directory_09]

# Used to store the merged data
merged_data = defaultdict(list)

# Iterate through subdirectories
for dir_path in sub_dirs:
    for file_name in os.listdir(dir_path):
        if file_name.endswith('.txt'):
            year_page, page_part = file_name.split('_')
            year = '19' + year_page[:2]
            page = int(year_page[2:])
            part = int(page_part.split('.')[0])

            file_path = os.path.join(dir_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()

            # Store file content
            merged_data[year].append((page, part, file_name, content))

# Sort by year, page number, and part number, and merge content
output_directory = '/content/personal_project/merged_files'
os.makedirs(output_directory, exist_ok=True)

for year, files in sorted(merged_data.items()):
    year_dir = os.path.join(output_directory, year)
    os.makedirs(year_dir, exist_ok=True)

    sorted_files = sorted(files, key=lambda x: (x[0], x[1]))

    for _, _, file_name, content in sorted_files:
        output_file_path = os.path.join(year_dir, file_name)
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(content)

print('File merging and sorting completed!')


In [ ]:
import os

# Specify the folder path
folder_path = '/content/personal_project/merged_files/1915'

# Initialize a string to store file content
all_content = ""

# Get and sort all .txt files in the folder
files = sorted([f for f in os.listdir(folder_path) if f.endswith('.txt')])

# Iterate through all .txt files in the folder
for filename in files:
    file_path = os.path.join(folder_path, filename)
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        all_content += content + " "  # Merge into the total content

# Split the merged content into chunks of 300 characters
def split_into_chunks(text, chunk_size=300):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

# Split the merged content
segment_1915 = split_into_chunks(all_content, chunk_size=300)


In [ ]:
print(len(segment_1915))

In [ ]:
# Data filtering
from transformers import DebertaModel, DebertaTokenizer
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the pre-trained DeBERTa model and tokenizer
model_name = 'microsoft/deberta-base'
tokenizer = DebertaTokenizer.from_pretrained(model_name)
model = DebertaModel.from_pretrained(model_name)

# Define the word to compare with
female_word = '女性'  # "female" in Chinese

# Function to get the text vector
def get_segment_vector(segment, tokenizer, model):
    inputs = tokenizer(segment, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    # Get the average of all token vectors as the text vector
    segment_vector = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return segment_vector

# Get the vector for the word "female"
female_vector = get_segment_vector(female_word, tokenizer, model)

# Calculate the similarity between each text and "female" and filter out texts with similarity less than 0.8
filtered_texts = []
similarities = []
for idx, segment in enumerate(segment_1915):
    segment_vector = get_segment_vector(segment.strip(), tokenizer, model)
    similarity = cosine_similarity(female_vector, segment_vector)[0][0]
    similarities.append(similarity)
    print(f"Text: {segment.strip()}\nSimilarity with 'female': {similarity}\n")

    if similarity >= 0.8:
        filtered_texts.append(segment.strip())

    # Print a progress message every 500 texts processed
    if idx % 500 == 0:
        print(f"Processed {idx + 1} texts so far")

# Output the final dataset size
final_size = len(filtered_texts)
print(f"Final dataset size: {final_size}")

# Name the filtered dataset
segment_1915 = filtered_texts


In [ ]:
import re

# Data cleaning function
def preprocess_data(text):
    # Use regular expressions to remove special characters and numbers
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', '', text)  # Remove whitespace
    # Return the preprocessed text
    return text

# Clean the text data in segment_1915
segment_1915_clean = []
for segment in segment_1915:
    data = preprocess_data(segment)
    segment_1915_clean.append(data)

# Save the cleaned data to a text file
with open('clean_segment_1915.txt', 'w', encoding='utf-8') as file:
    for data in segment_1915_clean:
        file.write(data + '\n')

print("Cleaned data saved successfully!")

In [ ]:
print(segment_1915_clean[0])